In [1]:
pip install cvxpy

Note: you may need to restart the kernel to use updated packages.


In [13]:
import numpy as np
import cvxpy as cp
from sklearn.covariance import GraphicalLassoCV
from scipy.linalg import svd
import pickle
import scipy.linalg
import Util
import os

In [14]:
def estimateur_MTP2(A ,k = 1e-6):
    T,N = A.shape
    
    S = np.cov(A.T)

    # Création de la variable de matrice symétrique
    K = cp.Variable((N, N), symmetric=True)

    # Maximun de Vraisemblance
    K_m = cp.Maximize(cp.log_det(K) - cp.trace(S @ K))

    # Les contraintes
    constraints = [K >> k]  # K doit être définie positive
    constraints += [K[i, j] <= 0 for i in range(N) for j in range(N) if i != j]
    constraints += [cp.diag(K) >= cp.abs(K).sum(axis=1)]

    # Formulation du problème
    prob = cp.Problem(K_m, constraints)

    # Résoudre le problème
    prob.solve()

    # Récupérer la matrice de précision optimisée
    if prob.status not in ["infeasible", "unbounded"]:
        return K.value
    else:
        # Handle the case where the problem is infeasible or unbounded
        print(f"Optimization was {prob.status} at this iteration")
        return None

In [15]:
def GLASSO(S): 
    model = GraphicalLassoCV(cv=3)
    model.fit(S)
    
    return np.linalg.inv(model.get_precision())

In [92]:
def clime(A):
    T,N = A.shape
    S = np.cov(A.T)
    Theta = cp.Variable((N, N))
    lambda_val = 0.5

    # Fonction objectif : norme l1 de Theta
    objective = cp.Minimize(cp.norm(Theta, 1))

    # Contraintes selon la définition de CLIME
    constraints = [cp.abs(S @ Theta - np.eye(N)) <= lambda_val]

    # Définition du problème
    prob = cp.Problem(objective, constraints)

    # Résolution du problème
    prob.solve()

    # Retourner la solution
    return Theta.value

##NLS dans R##

for (h in 1:360) {
    x <- data_r[h,,] - apply(data_r[h,,], 2, mean)
    NLS_cov <- nlshrink_cov(x, k=1)
    
    # Define the path for saving using paste0 for dynamic file names
    file_path <- paste0("C:/Users/lyhen/Downloads/MTP2/cov_NLS/cov_h", h, ".npy")
    
    # Save the cov matrix as a .npy file
    np$save(file_path, NLS_cov)
}


##LS dans R##

for (h in 1:360) {
    x <- data_r[h,,] - apply(data_r[h,,], 2, mean)
    LS_cov <- linshrink_cov(x, 0)
    
    # Define the path for saving using paste0 for dynamic file names
    file_path <- paste0("C:/Users/lyhen/Downloads/MTP2/cov_LS/cov_h", h, ".npy")
    
    # Save the cov matrix as a .npy file
    np$save(file_path, LS_cov)
}


##POET(5) dans R##

for (h in (1:360)){
    x <- data_r[h,,] - apply(data_r[h,,],2,mean)
    Y <- t(x)
    
    res <- POET(Y,5)
    cov <- res$SigmaY
    
    file_path <- paste0("C:/Users/lyhen/Downloads/MTP2/cov_POET5/cov_h", h, ".npy")
    np$save(file_path, cov)
}

Ensuit on modifie selon le cas que l'on souhaite

## test

### T = 25, N = 100

In [87]:
with open('pastRets/{}_{}_pastRets.pkl'.format(25,100), 'rb') as f:
    pastRets = pickle.load(f)
    assert len(pastRets) == 360

In [88]:
with open('outRets/{}_outRets.pkl'.format(100), 'rb') as f:
    outRets = pickle.load(f)

In [ ]:
estimator_portfolio_return = []

for h in range(360):
    cov = estimateur_MTP2(pastRets[h])
    # Get optimal portfolio weights.
    w = Util.optimal_weights(cov)
    # Calculate the return of the portfolio with given weights.
    curret = Util.retConstShare(outRets[h], w)
    estimator_portfolio_return.append(curret)
    
print(Util.get_IR(estimator_portfolio_return))

In [93]:
estimator_portfolio_return = []

for h in range(360):
    cov = clime(pastRets[h])
    # Get optimal portfolio weights.
    w = Util.optimal_weights(cov)
    # Calculate the return of the portfolio with given weights.
    curret = Util.retConstShare(outRets[h], w)
    estimator_portfolio_return.append(curret)
    
print(Util.get_IR(estimator_portfolio_return))

SolverError: Solver 'ECOS' failed. Try another solver, or solve with verbose=True for more information.

In [79]:
estimator_portfolio_return = []

for h in range(360):
    cov = GLASSO(pastRets[h])
    # Get optimal portfolio weights.
    w = Util.optimal_weights(cov)
    # Calculate the return of the portfolio with given weights.
    curret = Util.retConstShare(outRets[h], w)
    estimator_portfolio_return.append(curret)
    
print(Util.get_IR(estimator_portfolio_return))

C:\Users\lyhen\anaconda3\Lib\site-packages\numpy\core\_methods.py:236: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\lyhen\anaconda3\Lib\site-packages\numpy\core\_methods.py:236: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\lyhen\anaconda3\Lib\site-packages\numpy\core\_methods.py:236: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\lyhen\anaconda3\Lib\site-packages\sklearn\covariance\_graph_lasso.py:159: RuntimeWarning: overflow encountered in multiply
  precision_[indices != idx, idx] = -precision_[idx, idx] * coefs
C:\Users\lyhen\anaconda3\Lib\site-packages\sklearn\covariance\_graph_lasso.py:160: RuntimeWarning: overflow encountered in multiply
  precision_[idx, indices != idx] = -precision_[idx, idx] * coefs
C:\Users\lyhen\anaconda3\Lib\site-packages\numpy\core\_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return um

C:\Users\lyhen\anaconda3\Lib\site-packages\sklearn\covariance\_graph_lasso.py:159: RuntimeWarning: overflow encountered in multiply
  precision_[indices != idx, idx] = -precision_[idx, idx] * coefs
C:\Users\lyhen\anaconda3\Lib\site-packages\sklearn\covariance\_graph_lasso.py:160: RuntimeWarning: overflow encountered in multiply
  precision_[idx, indices != idx] = -precision_[idx, idx] * coefs
C:\Users\lyhen\anaconda3\Lib\site-packages\numpy\core\_methods.py:49: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
C:\Users\lyhen\anaconda3\Lib\site-packages\numpy\core\_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
C:\Users\lyhen\anaconda3\Lib\site-packages\numpy\core\_methods.py:236: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\lyhen\anaconda3\Lib\site-packages\numpy\core\_methods.py:236: RuntimeWa

C:\Users\lyhen\anaconda3\Lib\site-packages\numpy\core\_methods.py:236: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\lyhen\anaconda3\Lib\site-packages\numpy\core\_methods.py:236: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\lyhen\anaconda3\Lib\site-packages\numpy\core\_methods.py:236: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\lyhen\anaconda3\Lib\site-packages\numpy\core\_methods.py:236: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\lyhen\anaconda3\Lib\site-packages\numpy\core\_methods.py:236: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\lyhen\anaconda3\Lib\site-packages\numpy\core\_methods.py:236: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\lyhen\anaconda3\Lib\site-packages\numpy\core\_met

C:\Users\lyhen\anaconda3\Lib\site-packages\sklearn\covariance\_graph_lasso.py:159: RuntimeWarning: invalid value encountered in multiply
  precision_[indices != idx, idx] = -precision_[idx, idx] * coefs
C:\Users\lyhen\anaconda3\Lib\site-packages\sklearn\covariance\_graph_lasso.py:160: RuntimeWarning: invalid value encountered in multiply
  precision_[idx, indices != idx] = -precision_[idx, idx] * coefs
C:\Users\lyhen\anaconda3\Lib\site-packages\numpy\core\_methods.py:236: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\lyhen\anaconda3\Lib\site-packages\numpy\core\_methods.py:236: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\lyhen\anaconda3\Lib\site-packages\numpy\core\_methods.py:236: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\lyhen\anaconda3\Lib\site-packages\numpy\core\_methods.py:236: RuntimeWarning: invalid value encountered in subtrac

C:\Users\lyhen\anaconda3\Lib\site-packages\numpy\core\_methods.py:236: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\lyhen\anaconda3\Lib\site-packages\numpy\core\_methods.py:236: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\lyhen\anaconda3\Lib\site-packages\numpy\core\_methods.py:236: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\lyhen\anaconda3\Lib\site-packages\sklearn\covariance\_graph_lasso.py:159: RuntimeWarning: overflow encountered in multiply
  precision_[indices != idx, idx] = -precision_[idx, idx] * coefs
C:\Users\lyhen\anaconda3\Lib\site-packages\sklearn\covariance\_graph_lasso.py:160: RuntimeWarning: overflow encountered in multiply
  precision_[idx, indices != idx] = -precision_[idx, idx] * coefs
C:\Users\lyhen\anaconda3\Lib\site-packages\numpy\core\_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return um

...

C:\Users\lyhen\anaconda3\Lib\site-packages\numpy\core\_methods.py:236: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\lyhen\anaconda3\Lib\site-packages\sklearn\covariance\_graph_lasso.py:159: RuntimeWarning: overflow encountered in multiply
  precision_[indices != idx, idx] = -precision_[idx, idx] * coefs
C:\Users\lyhen\anaconda3\Lib\site-packages\sklearn\covariance\_graph_lasso.py:160: RuntimeWarning: overflow encountered in multiply
  precision_[idx, indices != idx] = -precision_[idx, idx] * coefs
C:\Users\lyhen\anaconda3\Lib\site-packages\numpy\core\_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
C:\Users\lyhen\anaconda3\Lib\site-packages\sklearn\covariance\_graph_lasso.py:159: RuntimeWarning: overflow encountered in multiply
  precision_[indices != idx, idx] = -precision_[idx, idx] * coefs
C:\Users\lyhen\anaconda3\Lib\site-packages\sklearn\covariance\_

C:\Users\lyhen\anaconda3\Lib\site-packages\numpy\core\_methods.py:236: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\lyhen\anaconda3\Lib\site-packages\numpy\core\_methods.py:236: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\lyhen\anaconda3\Lib\site-packages\numpy\core\_methods.py:236: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\lyhen\anaconda3\Lib\site-packages\sklearn\covariance\_graph_lasso.py:159: RuntimeWarning: invalid value encountered in multiply
  precision_[indices != idx, idx] = -precision_[idx, idx] * coefs
C:\Users\lyhen\anaconda3\Lib\site-packages\sklearn\covariance\_graph_lasso.py:160: RuntimeWarning: invalid value encountered in multiply
  precision_[idx, indices != idx] = -precision_[idx, idx] * coefs
C:\Users\lyhen\anaconda3\Lib\site-packages\numpy\core\_methods.py:236: RuntimeWarning: invalid value encountered in subtrac

C:\Users\lyhen\anaconda3\Lib\site-packages\numpy\core\_methods.py:236: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\lyhen\anaconda3\Lib\site-packages\numpy\core\_methods.py:236: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\lyhen\anaconda3\Lib\site-packages\sklearn\covariance\_graph_lasso.py:159: RuntimeWarning: overflow encountered in multiply
  precision_[indices != idx, idx] = -precision_[idx, idx] * coefs
C:\Users\lyhen\anaconda3\Lib\site-packages\sklearn\covariance\_graph_lasso.py:160: RuntimeWarning: overflow encountered in multiply
  precision_[idx, indices != idx] = -precision_[idx, idx] * coefs
C:\Users\lyhen\anaconda3\Lib\site-packages\numpy\core\_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
C:\Users\lyhen\anaconda3\Lib\site-packages\numpy\core\_methods.py:236: RuntimeWarning: invalid value enc

(12.321983759792147, 13.809259269124677, 0.8922986758125511)


C:\Users\lyhen\anaconda3\Lib\site-packages\numpy\core\_methods.py:236: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)


In [62]:
estimator_portfolio_return = []

for h in range(360):
    cov = np.load("cov_LS/cov_h{}.npy".format(h+1))
    w = Util.optimal_weights(cov)
    curret = Util.retConstShare(outRets[h], w)
    estimator_portfolio_return.append(curret)

In [63]:
print(Util.get_IR(estimator_portfolio_return))

(9.884623000499307, 13.806762754201884, 0.7159261860634976)


In [64]:
estimator_portfolio_return = []

for h in range(360):
    cov = np.load("cov_NLS/cov_h{}.npy".format(h+1))
    w = Util.optimal_weights(cov)
    curret = Util.retConstShare(outRets[h], w)
    estimator_portfolio_return.append(curret)

In [65]:
print(Util.get_IR(estimator_portfolio_return))

(13.626700057782408, 19.650358758579667, 0.6934580800888822)


In [66]:
estimator_portfolio_return = []

for h in range(360):
    cov = np.load("cov_POET3/cov_h{}.npy".format(h+1))
    w = Util.optimal_weights(cov)
    curret = Util.retConstShare(outRets[h], w)
    estimator_portfolio_return.append(curret)

In [67]:
print(Util.get_IR(estimator_portfolio_return))

(10.036335148418916, 13.029097612523438, 0.7703016315398613)


In [68]:
estimator_portfolio_return = []

for h in range(360):
    cov = np.load("cov_POET5/cov_h{}.npy".format(h+1))
    w = Util.optimal_weights(cov)
    curret = Util.retConstShare(outRets[h], w)
    estimator_portfolio_return.append(curret)

In [69]:
print(Util.get_IR(estimator_portfolio_return))

(10.72356757358823, 12.890734192911843, 0.8318818317954876)


### T = 50 , N = 50

In [82]:
with open('pastRets/{}_{}_pastRets.pkl'.format(50,50), 'rb') as f:
    pastRets = pickle.load(f)
    assert len(pastRets) == 360

In [83]:
with open('outRets/{}_outRets.pkl'.format(50), 'rb') as f:
    outRets = pickle.load(f)

In [ ]:
estimator_portfolio_return = []

for h in range(360):
    cov = estimateur_MTP2(pastRets[h])
    # Get optimal portfolio weights.
    w = Util.optimal_weights(cov)
    # Calculate the return of the portfolio with given weights.
    curret = Util.retConstShare(outRets[h], w)
    estimator_portfolio_return.append(curret)
    
print(Util.get_IR(estimator_portfolio_return))

In [84]:
estimator_portfolio_return = [] # lambda = 

for h in range(360):
    cov = clime(pastRets[h])
    # Get optimal portfolio weights.
    w = Util.optimal_weights(cov)
    # Calculate the return of the portfolio with given weights.
    curret = Util.retConstShare(outRets[h], w)
    estimator_portfolio_return.append(curret)
    
print(Util.get_IR(estimator_portfolio_return))

AttributeError: 'NoneType' object has no attribute 'shape'

In [53]:
estimator_portfolio_return = []

for h in range(360):
    filepath = "50cov_NLS/cov_h{}.npy".format(h+1) 
    if os.path.exists(filepath):# we have some issue for certain h so we lack of those values
        cov = np.load(filepath)
        w = Util.optimal_weights(cov)
        curret = Util.retConstShare(outRets[h], w)
        estimator_portfolio_return.append(curret)
    
print(Util.get_IR(estimator_portfolio_return))

(nan, nan, nan)


C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [5]:
estimator_portfolio_return = []

for h in range(360):
    cov = np.load("50cov_LS/cov_h{}.npy".format(h+1))
    w = Util.optimal_weights(cov)
    curret = Util.retConstShare(outRets[h], w)
    estimator_portfolio_return.append(curret)
    
print(Util.get_IR(estimator_portfolio_return))

(8.108071972722955, 14.580473258133264, 0.5560911384135024)


In [6]:
estimator_portfolio_return = []

for h in range(360):
    cov = np.load("50cov_POET3/cov_h{}.npy".format(h+1))
    w = Util.optimal_weights(cov)
    curret = Util.retConstShare(outRets[h], w)
    estimator_portfolio_return.append(curret)
    
print(Util.get_IR(estimator_portfolio_return))

(7.727593443207459, 14.020184342557446, 0.5511763079855378)


In [7]:
estimator_portfolio_return = []

for h in range(360):
    cov = np.load("50cov_POET5/cov_h{}.npy".format(h+1))
    w = Util.optimal_weights(cov)
    curret = Util.retConstShare(outRets[h], w)
    estimator_portfolio_return.append(curret)
    
print(Util.get_IR(estimator_portfolio_return))

(9.459422132669578, 13.880828961709254, 0.6814738628913101)


In [6]:
estimator_portfolio_return = []

for h in range(360):
    cov = GLASSO(pastRets[h])
    # Get optimal portfolio weights.
    w = Util.optimal_weights(cov)
    # Calculate the return of the portfolio with given weights.
    curret = Util.retConstShare(outRets[h], w)
    estimator_portfolio_return.append(curret)
    
print(Util.get_IR(estimator_portfolio_return))

C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: R

C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\sklearn\covariance\_graph_lasso.py:272: RuntimeWarning: overflow encountered in multiply
  precision_[indices != idx, idx] = -precision_[idx, idx] * coefs
C:\Users\Neo\anaconda3\Lib\site-packages\sklearn\covariance\_graph_lasso.py:273: RuntimeWarning: overflow encountered in multiply
  precision_[idx, indices != idx] = -precision_[idx, idx] * coefs
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\sklearn\covariance\_graph_lasso.py:272: RuntimeWarning: invalid value encountered in multiply
  precis

C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\sklearn\covariance\_graph_lasso.py:272: RuntimeWarning: invalid value encountered in multiply
  precision_[indices != idx, idx] = -precision_[idx, idx] * coefs
C:\Users\Neo\anaconda3\L

C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\sklearn\covariance\_graph_lasso.py:272: RuntimeWarning: invalid value encountered in multiply
  precision_[indices != idx, idx] = -precision_[idx, idx] * coefs
C:\Users\Neo\anaconda3\Lib\site-packages\sklearn\covariance\_graph_lasso.py:273: RuntimeWarning: invalid value encountered in multiply
  precision_[idx, indices != idx] = -precision_[idx, idx] * coefs
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\sklearn\covariance\_graph_lasso.py:272: RuntimeWarning: invalid value encountered in multiply
  precision_[indices != idx, idx] = -precision_[idx, idx] * coefs
C:\Users\Neo\anaconda3\Lib\site-packages\sklearn\covariance\_graph_lasso.py:273: Runtim

C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\sklearn\covariance\_graph_las

...

C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\sklearn\covariance\_graph_las

C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\sklearn\covariance\_graph_lasso.py:272: RuntimeWarning: invalid value encountered in multiply
  precision_[indices != idx, idx] = -precision_[idx, idx] * coefs
C:\Users\Neo\anaconda3\Lib\site-packages\sklearn\covariance\_graph_lasso.py:273: RuntimeWarning: invalid value encountered in multiply
  precision_[idx, indices != idx] = -precision_[idx, idx] * coefs
C:\Users\Neo\anaconda3\Lib\site-packages\sklearn\covariance\_graph_lasso.py:272: RuntimeWarning: overflow encountered in multiply
  precision_[indices != idx, idx] = -precision_[idx, idx] * coefs
C:\Users\Neo\anaconda3\Lib\site-packages\sklearn\covariance\_graph_lasso.py:273: RuntimeWarning: overflow encountered in multiply
  precision_[idx, indices != idx] = -precision_[idx, idx] * coefs
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core

C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\sklearn\covariance\_graph_lasso.py:272: RuntimeWarning: overflow encountered in multiply
  precision_[indices != idx, idx] = -precision_[idx, idx] * coefs
C:\Users\Neo\anaconda3\Lib\site-packages\sklearn\covariance\_graph_lasso.py:273: RuntimeWarning: overflow encountered in multiply
  precision_[idx, indices != idx] = -precision_[idx, idx] * coefs
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr

(10.687317334432835, 14.18849931674757, 0.7532380342590514)


C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)


### T = 100, N = 25

In [44]:
with open('pastRets/{}_{}_pastRets.pkl'.format(100,25), 'rb') as f:
    pastRets = pickle.load(f)
    assert len(pastRets) == 360

In [45]:
with open('outRets/{}_outRets.pkl'.format(25), 'rb') as f:
    outRets = pickle.load(f)

In [ ]:
estimator_portfolio_return = []

for h in range(360):
    cov = estimateur_MTP2(pastRets[h])
    # Get optimal portfolio weights.
    w = Util.optimal_weights(cov)
    # Calculate the return of the portfolio with given weights.
    curret = Util.retConstShare(outRets[h], w)
    estimator_portfolio_return.append(curret)
    
print(Util.get_IR(estimator_portfolio_return))

In [48]:
estimator_portfolio_return = [] #lambda = T/N

for h in range(360):
    cov = clime(pastRets[h])
    # Get optimal portfolio weights.
    w = Util.optimal_weights(cov)
    # Calculate the return of the portfolio with given weights.
    curret = Util.retConstShare(outRets[h], w)
    estimator_portfolio_return.append(curret)
    
print(Util.get_IR(estimator_portfolio_return))

(8.797984543585258, 16.982935572676958, 0.5180485144005327)


In [71]:
estimator_portfolio_return = []

for h in range(360):
    cov = np.load("25cov_LS/cov_h{}.npy".format(h+1))
    w = Util.optimal_weights(cov)
    curret = Util.retConstShare(outRets[h], w)
    estimator_portfolio_return.append(curret)

In [72]:
print(Util.get_IR(estimator_portfolio_return))

(9.946097836403197, 15.766276475778202, 0.6308463416637041)


In [83]:
estimator_portfolio_return = []

for h in range(360):
    cov = np.load("25cov_NLS/cov_h{}.npy".format(h+1))
    w = Util.optimal_weights(cov)
    curret = Util.retConstShare(outRets[h], w)
    estimator_portfolio_return.append(curret)
    
print(Util.get_IR(estimator_portfolio_return))

(9.300924314921, 16.003646302732438, 0.581175323359463)


In [73]:
estimator_portfolio_return = []

for h in range(360):
    cov = np.load("25cov_POET5/cov_h{}.npy".format(h+1))
    w = Util.optimal_weights(cov)
    curret = Util.retConstShare(outRets[h], w)
    estimator_portfolio_return.append(curret)
    
print(Util.get_IR(estimator_portfolio_return))

(7.6330374882988306, 17.050244285341478, 0.44767906902431565)


In [74]:
estimator_portfolio_return = []

for h in range(360):
    cov = np.load("25cov_POET3/cov_h{}.npy".format(h+1))
    w = Util.optimal_weights(cov)
    curret = Util.retConstShare(outRets[h], w)
    estimator_portfolio_return.append(curret)
    
print(Util.get_IR(estimator_portfolio_return))

(8.76092021514503, 15.88771191701459, 0.5514274340386748)


In [7]:
estimator_portfolio_return = []

for h in range(360):
    cov = GLASSO(pastRets[h])
    # Get optimal portfolio weights.
    w = Util.optimal_weights(cov)
    # Calculate the return of the portfolio with given weights.
    curret = Util.retConstShare(outRets[h], w)
    estimator_portfolio_return.append(curret)
    
print(Util.get_IR(estimator_portfolio_return))

C:\Users\Neo\anaconda3\Lib\site-packages\sklearn\covariance\_graph_lasso.py:272: RuntimeWarning: invalid value encountered in multiply
  precision_[indices != idx, idx] = -precision_[idx, idx] * coefs
C:\Users\Neo\anaconda3\Lib\site-packages\sklearn\covariance\_graph_lasso.py:273: RuntimeWarning: invalid value encountered in multiply
  precision_[idx, indices != idx] = -precision_[idx, idx] * coefs
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\sklearn\covariance\_graph_lasso.py:272: RuntimeWarning: invalid value encountered in multipl

C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: R

C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\sklearn\covariance\_graph_lasso.py:272: RuntimeWarning: invalid value encountered in multiply
  precision_[indices != idx, idx] = -precision_[idx, idx] * coefs
C:\Users\Neo\anaconda3\Lib\site-packages\sklearn\covariance\_graph_lasso.py:273: RuntimeWarning: invalid value encountered in multiply
  precision_[idx, indices != idx] = -precision_[idx, idx] * coefs
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asan

C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\sklearn\covariance\_graph_lasso.py:272: RuntimeWarning: invalid value encountered in multiply
  precision_[indices != idx, idx] = -precision_[idx, idx] * coefs
C:\Users\Neo\anaconda3\L

C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\sklearn\covariance\_graph_lasso.py:272: RuntimeWarning: invalid value encountered in multiply
  precision_[indices != idx, idx] = -precision_[idx, idx] * coefs
C:\Users\Neo\anaconda3\Lib\site-packages\sklearn\covariance\_graph_lasso.py:273: RuntimeWarning: invalid value encountered in multiply
  precision_[idx, indices != idx] = -precision_[idx, idx] * coefs
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asan

...

C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: R

C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\sklearn\covariance\_graph_lasso.py:272: RuntimeWarning: invalid value encountered in multiply
  precision_[indices != idx, idx] = -precision_[idx, idx] * coefs
C:\Users\Neo\anaconda3\Lib\site-packages\sklearn\covariance\_graph_lasso.py:273: RuntimeWarning: invalid value encountered in multiply
  precision_[idx, indices != idx] = -precisi

C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\sklearn\covariance\_graph_lasso.py:272: RuntimeWarning: invalid value encountered in multiply
  precision_[indices != idx, idx] = -precision_[idx, idx] * coefs
C:\Users\Neo\anaconda3\Lib\site-packages\sklearn\covariance\_graph_lasso.py:273: RuntimeWarning: invalid value encountered in multiply
  precision_[idx, indices != idx] = -precision_[idx, idx] * coefs
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asan

(10.538358019404525, 15.161754792791582, 0.695061895105626)


C:\Users\Neo\anaconda3\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
